In [ ]:
# Este codigo lee modelos metabolicos SBML y extrae todas las reacciones y sus genes asociados,
# para analizar las reglas logicas GPR y calcular el numero minimo de deleciones geneticas para perder la reaccion
# Genera un CSV sin considerar las reacciones que NO tienen GPR asociado (cada fila es una reaccion de cada modelo)


### INSTALACION Y MONTAJE
!pip install -q "cobra==0.29.0" "optlang==1.8.3" "python-libsbml==5.20.2" "swiglpk>=5.0.5"

# -q quiet
# cobra: analisis modelos metabolicos
# optlang: solvers optimizacion
# libsbml: manejo archivos sbml
# swiglpk: solver problema lineal


## Conectar Colab con Drive
from google.colab import drive
drive.mount('/content/drive')

### IMPORTS Y CONFIGURACION
from cobra import io
import os, glob, pandas as pd, logging, re
from tqdm.auto import tqdm
from itertools import combinations
import warnings
from datetime import datetime

# io: manejo modelos sbml (Cobra)
# os: manejo rutas y directorios
# glob: buscar archivos con patrones
# panda: manejo de tablas como DataFrame
# logging: silenciar logs de Cobra
# re: manejo de patrones (al, chon, doc...)
# tqdm: barra progreso visual
# combinations: genera combinaciones de genes (delec min)
# warnings: silenciar advertencias
# datetime: para nombrar carpetas generadas segun fecha y hora (evita sobreescribir)

## Silenciar logs y warnings:
warnings.filterwarnings('ignore')
logging.getLogger('cobra').setLevel(logging.CRITICAL)

## Rutas
CARPETA_MODELOS = "/content/drive/MyDrive/MASH_primavera_2025/modelos_sbml" # 211 modelos sbml
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
CARPETA_SALIDA = f"/content/drive/MyDrive/MASH_primavera_2025/out/run_{timestamp}"
os.makedirs(CARPETA_SALIDA, exist_ok=True)
SALIDA_CSV = os.path.join(CARPETA_SALIDA, "reacciones_completo.csv") # acá guardo CSV

# Obtener lista de modelos en orden alfabetico
paths = sorted(glob.glob(os.path.join(CARPETA_MODELOS, "*.sbml")))
# paths = sorted(glob.glob(os.path.join(CARPETA_MODELOS, "*.sbml")))[:2]  # Solo primeros 2

# Mostrar
print(f"Modelos encontrados: {len(paths)}")
print(f"Carpeta salida: {CARPETA_SALIDA}\n")

### FUNCIONES
# Normalizar GPR (eliminar espacios extra)
def normaliza_gpr(gpr: str) -> str:
    return re.sub(r"\s+", " ", (gpr or "").strip())

# Convertir reaccion a string de estequiometria
def to_esteq_str(rxn) -> str:
    try:
        return rxn.reaction
    except:
        lhs = " + ".join(f"{-c} {m.id}" for m,c in rxn.metabolites.items() if c < 0)
        rhs = " + ".join(f"{c} {m.id}" for m,c in rxn.metabolites.items() if c > 0)
        arrow = "<->" if rxn.reversibility else "->"
        return f"{lhs} {arrow} {rhs}"

# Parsear GPR a clausulas DNF (OR de ANDs)
def dnf_clauses(gpr: str):
    if not gpr:
        return []
    expr = gpr.lower()
    partes, nivel, buf, i = [], 0, [], 0

    while i < len(expr):
        ch = expr[i]
        if ch == "(":
            nivel += 1
            buf.append(ch)
        elif ch == ")":
            nivel = max(0, nivel-1)
            buf.append(ch)
        elif nivel == 0 and expr[i:i+4] == " or ":
            partes.append("".join(buf).strip())
            buf = []
            i += 3
        else:
            buf.append(ch)
        i += 1

    if buf:
        partes.append("".join(buf).strip())

    clausulas = []
    for p in partes:
        p = p.strip()
        if p.startswith("(") and p.endswith(")"):
            p = p[1:-1].strip()
        genes = [g.strip("() ") for g in p.split(" and ") if g.strip()]
        clausulas.append(set(genes))
    return clausulas

# Calcular minimo de genes a eliminar para inactivar reaccion
def min_deleciones_para_inactivar(gpr: str):
    if not gpr:
        return None
    clauses = dnf_clauses(gpr)
    if not clauses:
        return None

    lb = min(len(c) for c in clauses if c)
    universo = sorted(set().union(*clauses))
    LIM = min(6, len(universo))  # combinaciones de 6 genes maximo, si no se satura (OJO)

    for k in range(lb, LIM+1):
        for combo in combinations(universo, k):
            S = set(combo)
            if all(len(c & S) >= 1 for c in clauses):
                return k
    return lb


### GENERAR CSV
cols = ["modelo_id", "sitio", "rxn_id", "rxn_nombre", "estequiometria",
        "gpr", "genes", "subsystem", "ec", "min_deleciones_gpr"]

# Crear archivo con header
with open(SALIDA_CSV, "w") as f:
    f.write(",".join(cols) + "\n")

contador_total = 0
contador_sin_gpr = 0

# Procesar cada modelo
for ruta in tqdm(paths, desc="Modelos", unit="modelo"):
    try:
        modelo_id = os.path.splitext(os.path.basename(ruta))[0]
        # Extraer sitio del nombre (primeras letras antes de numeros)
        sitio = re.match(r'^([a-z]+)', modelo_id).group(1) if re.match(r'^([a-z]+)', modelo_id) else "desconocido"

        # CARGAR MODELO
        m = io.read_sbml_model(ruta)
        filas = []

        for rxn in m.reactions:
            # Filtrar reacciones sin GPR
            if not rxn.gene_reaction_rule or not rxn.genes:
                contador_sin_gpr += 1
                continue

            contador_total += 1

            gpr = normaliza_gpr(rxn.gene_reaction_rule)
            genes = ",".join(sorted(g.id for g in rxn.genes))
            subsystem = getattr(rxn, "subsystem", "")

            # Buscar EC number
            ec = None
            if isinstance(rxn.annotation, dict):
                ec = rxn.annotation.get("ec-code") or rxn.annotation.get("ec_number")
            if not ec:
                match = re.search(r'\d+\.\d+\.\d+\.\d+', rxn.id)
                if match:
                    ec = match.group(0)

            filas.append({
                "modelo_id": modelo_id,
                "sitio": sitio,
                "rxn_id": rxn.id,
                "rxn_nombre": rxn.name,
                "estequiometria": to_esteq_str(rxn),
                "gpr": gpr,
                "genes": genes,
                "subsystem": subsystem if subsystem else "",
                "ec": ec if ec else "",
                "min_deleciones_gpr": min_deleciones_para_inactivar(gpr)
            })

        # Guardar en CSV (append)
        pd.DataFrame(filas).to_csv(SALIDA_CSV, mode="a", header=False,
                                   index=False, quoting=1)

    except Exception as e:
        print(f"\nError en {modelo_id}: {e}") # printear modelos con error si esq hay

print(f"\nCSV generado: {SALIDA_CSV}") # printear carpeta csv
print(f"Reacciones guardadas: {contador_total:,}") # cuantas rxns proceso
print(f"Reacciones sin GPR eliminadas: {contador_sin_gpr:,}")

### VERIFICACION
# Leer solo primeras 1000 filas para verificar
df_check = pd.read_csv(SALIDA_CSV, sep=',', encoding='latin-1',
                       quotechar='"', quoting=0, on_bad_lines='warn',
                       engine='python', nrows=1000)

print(f"\nVerificacion (primeras 1000 filas):")
print(f"Columnas: {len(df_check.columns)}")
print(f"Modelos en muestra: {df_check['modelo_id'].nunique()}")
print(f"Reacciones en muestra: {df_check['rxn_id'].nunique()}")

# Contar filas totales en archivo
with open(SALIDA_CSV, 'r', encoding='latin-1') as f:
    lineas_total = sum(1 for _ in f) - 1  # Restar header

print(f"\nFilas totales en CSV: {lineas_total:,}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.8/141.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.2/788.2 kB 40.0 MB/s eta 0:00:00
Mounted at /content/drive
Modelos encontrados: 211
Carpeta salida: /content/drive/MyDrive/MASH_primavera_2025/out/run_20251225_182334



Modelos:   0%|          | 0/211 [00:00<?, ?modelo/s]


CSV generado: /content/drive/MyDrive/MASH_primavera_2025/out/run_20251225_182334/reacciones_completo.csv
Reacciones guardadas: 231,292
Reacciones sin GPR eliminadas: 87,868

Verificacion (primeras 1000 filas):
Columnas: 10
Modelos en muestra: 1
Reacciones en muestra: 1000

Filas totales en CSV: 231,292
